# Revised case normalization for Winterthur 2019

Notes:

- 90 cases can be matched with the DB (if ignoring patientID), otherwise 57.

In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize_ksw_2019
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(
         os.path.join(ROOT_DIR, 'raw_data/Winterthur.xlsx'),
         'KSW',
         '2019',
         'Änderungen Winterthur 2019')

print(file_info)



FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/Winterthur.xlsx', hospital_name_db='KSW', year='2019', sheet='Änderungen Winterthur 2019')


In [3]:
revised_cases_df = normalize_ksw_2019(file_info)
revised_cases_df

2022-11-14 09:15:33.757 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize_ksw_2019:141 - Read 90 cases for KSW 2019
2022-11-14 09:15:33.761 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize_ksw_2019:170 - TYPES:
tranche                                                 string
datum                                                   string
case_id                                                 object
patient_id                                              object
kkik                                                    string
gender                                                  string
age_years                                                int64
bfs_code                                                string
duration_of_stay                                         int64
pflegetage neu                                          string
pccl                                                     int64
pccl neu                     

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,900655820,A5609A39945ACA88,W,82,7,900655820,K219,K219,M990,2,G46C,[L8914],[],[],[]
1,900662000,6DE2468436F495DF,M,71,2,900662000,K599,K599,M990,3,G72C,[F322],[F328],[],[]
2,900663300,0C7C6ADC089DA71C,W,71,13,900663300,J850,J850,M990,3,E77C,"[B965, J44]",[],[],[]
3,900744100,2695CA186FA0279E,M,78,5,900744100,T814,L022,M990,2,T61B,[],[],[],[]
4,900748030,BBD20634F770F9B3,W,68,4,900748030,K701,K701,M990,3,H60C,[F102],[F101],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,900581254,1067388,M,61,9,900581254,C492,C492,M990,0,I27C,[],[],[86421B:R:20190418],[834936:R:20190418]
86,900638893,0720597,W,52,16,900638893,C797,C797,M990,4,K03Z,"[K632, D508, L271, E46]",[],"[009B11::20190710, 99B711::20190710, 99B810::20190710, 890A32::20190717, 5451::20190710, ...]",[]
87,900708132,0229695,W,67,9,900708132,J690,J690,M990,2,E77F,"[F410, J91]",[],[],[]
88,900711481,0422858,W,47,7,900711481,C786,C786,M990,3,G12C,[],[],[5451::20190814],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted

cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.head()

2022-11-14 09:15:35.200 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:90 - Read 27903 rows from the DB, for the hospital 'KSW' in 2019


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,312427,900584428,K5733,"[D62, D648, E559, R391, F708, ...]",4523::20190417,"[451310::20190417, 990410::20190417, 4523::20190422, 451310::20190422]",M,75,0,0,7,0,01,2019-04-16,0,00,2019-04-23
1,312817,900538395,K8050,"[K8510, K8020, K298, E876, K290]",4516::20190219,"[887415::20190219, 887430::20190219, 5185::20190219, 5188::20190219]",M,55,0,0,5,0,01,2019-02-16,0,00,2019-02-21
2,313807,900530732,J4400,"[M1007, I480, E871, Y579, I1090]",8937::20190206,[],M,68,0,0,6,0,01,2019-02-06,0,00,2019-02-12
3,313898,900656399,C61,"[C772, C795, D62, N390, B965]",6022::20190807,"[399A11:R:20190807, 990410::20190807]",M,78,0,0,3,0,01,2019-08-07,0,00,2019-08-10
4,314039,900531410,J4409,"[J101, J9600, K590, Y579, E039, ...]",998425::20190207,[],M,47,0,0,12,0,01,2019-02-07,0,00,2019-02-19


In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-14 09:15:35.626 | INFO     | src.revised_case_normalization.notebook_functions.group:group:83 - Grouping 90 cases ...
2022-11-14 09:15:36.647 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:88 - Grouped 90 cases into: 90 revisions, 824 diagnoses rows, 414 procedure rows


In [7]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,312427,G48C,0.97,0.97,3,2022-12-31
1,312817,H41C,1.27,1.27,3,2022-12-31
2,313807,E65B,0.80,0.80,2,2022-12-31
3,313898,M09A,1.50,1.50,4,2022-12-31
4,314039,E65A,1.18,1.18,4,2022-12-31
...,...,...,...,...,...,...
85,339465,F75B,1.66,1.66,4,2022-12-31
86,339616,I09A,3.71,3.71,4,2022-12-31
87,339723,T60D,1.42,1.42,4,2022-12-31
88,340012,N01C,2.32,2.32,2,2022-12-31


In [8]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-14 09:15:36.665 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:249 - Trying to insert 90 cases into the 'Revisions' table ...
2022-11-14 09:15:37.249 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:275 - Deleted 90 rows from the "Revisions" table, which is about to be updated
2022-11-14 09:15:37.345 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:286 - Inserted 90 cases into the 'Revisions' table
2022-11-14 09:15:37.350 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:298 - Trying to insert 824 rows into the 'Diagnoses' table ...
2022-11-14 09:15:39.259 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:336 - Inserted 824 rows into the 'Diagnoses' table
2022-11-14 09:15:39.260 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:347 - Trying to insert 414 rows into the 'Procedures' ta